# 6.7 - Regresyon Problemleri ve Lineer Regresyon Modeli: Aşırı Uyumluluk ve Regularizasyon

# Ev fiyatları modeli
## Model 1: OLS Regresyonu
Bir önceki derste en iyi açıklama yüzdesine sahip model kullanılacaktır.<br>
Değişkenler: **["TotalBsmtSF", "OverallQual", "GrLivArea", "YearBuilt"]**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, Lasso, ElasticNet, RidgeCV, LassoCV, ElasticNetCV
    # https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html
    # https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html
    # https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html
    # https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html
    # https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html
    # https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNetCV.html
from sklearn.model_selection import KFold, cross_validate, cross_val_predict
    # https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html
    # https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html
    # https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html
import statsmodels.api as sm
from statsmodels.tools.eval_measures import mse, rmse

%matplotlib inline
sns.set(style = "whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)

title_font = {"family":"arial", "weight":"bold", "color":"darkred", "size":16}
axis_font = {"family":"sans", "weight":"normal", "color":"darkblue", "size":14}

house = pd.read_csv("house_prices.csv")

In [2]:
# Modelimizi oluşturalım:
x1 = house[["TotalBsmtSF", "OverallQual", "GrLivArea", "YearBuilt"]]
y1 = house["SalePrice"]

In [3]:
kFold = KFold(10, shuffle=True, random_state=42)
mae1, mse1, rmse1, mape1, trainR_mean, testR_mean = ([] for i in range(6))

for train, test in kFold.split(x1, y1): 
    x1_train = x1.iloc[train]
    x1_test = x1.iloc[test]
    y1_train = y1.iloc[train]
    y1_test = y1.iloc[test]
    
    # Eğitim verilerinin R-kare değeri
    x1_train = sm.add_constant(x1_train)
    results1_model = sm.OLS(y1_train, x1_train)
    results1_train = results1_model.fit()
    trainR_mean.append(results1_train.rsquared_adj)

    # Test verilerinin R-kare değeri
    x1_test = sm.add_constant(x1_test)
    results1_model = sm.OLS(y1_test, x1_test)
    results1_test = results1_model.fit()
    testR_mean.append(results1_test.rsquared_adj)

    y1_predict = results1_train.predict(x1_test)

    mae0 = mean_absolute_error(y1_test, y1_predict)
    mse0 = mse(y1_test, y1_predict)
    rmse0 = rmse(y1_test, y1_predict)
    mape0 = (abs((y1_test - y1_predict)) / y1_test).mean() * 100
    
    mae1.append(mae0)
    mse1.append(mse0)
    rmse1.append(rmse0)
    mape1.append(mape0)

print(f"Eğitim verilerinin ayarlanmış R-kare değeri : {np.array(trainR_mean).mean():.3f}")
print(f"Test verilerinin ayarlanmış R-kare değeri   : {np.array(testR_mean).mean():.3f}")
print("\n---------Test Kümesi Hata İstatistikleri---------")
print(f"Ortalama Mutlak Hata (MAE)        : {np.array(mae1).mean():.2f}")
print(f"Ortalama Kare Hata (MSE)          : {np.array(mse1).mean():.2f}")
print(f"Kök Ortalama Kare Hata (RMSE)     : {np.array(rmse1).mean():.2f}")
print(f"Ortalama Mutlak Yüzde Hata (MAPE) : %{np.array(mape1).mean():.2f}")

Eğitim verilerinin ayarlanmış R-kare değeri : 0.758
Test verilerinin ayarlanmış R-kare değeri   : 0.776

---------Test Kümesi Hata İstatistikleri---------
Ortalama Mutlak Hata (MAE)        : 25102.96
Ortalama Kare Hata (MSE)          : 1593105385.53
Kök Ortalama Kare Hata (RMSE)     : 39002.47
Ortalama Mutlak Yüzde Hata (MAPE) : %15.08


***
## Model 2: Ridge Regresyonu

In [4]:
cv_results2 = cross_validate(Ridge(), x1_train, y1_train, cv=5, 
                             return_train_score = True)
cv_results2

{'fit_time': array([0.00799417, 0.00599432, 0.00600028, 0.00599003, 0.00399518]),
 'score_time': array([0.00499773, 0.00199842, 0.00399518, 0.0030005 , 0.00300217]),
 'test_score': array([0.79177286, 0.77652249, 0.76823701, 0.77598943, 0.58026951]),
 'train_score': array([0.74367625, 0.74506846, 0.74523165, 0.74610478, 0.78621681])}

In [5]:
## yukarıdaki cv_results2'de, alpha değeri girmedik. 1 mi alıyor alpha'yı?

#ridge_reg = Ridge(alpha = 1000)
#ridge_reg.fit(x1_train, y1_train)

y_test_predict = cross_val_predict(Ridge(), x1_test, y1_test, cv=5)
## Burada, "x1_test, y1_test" çifti yerine neden "x1_train, y1_train" koymadık?
    ## (zaten train koyunca boyut hatası veriyor)

var1 = cv_results2["train_score"].mean()
var2 = cv_results2["test_score"].mean()

print(f"Eğitim verilerinin ayarlanmış R-kare değeri: {var1:.3f}")
print(f"Test verilerinin ayarlanmış R-kare değeri: {var2:.3f}")

ridge_CV = RidgeCV(alphas=np.logspace(-1, 5, 7)).fit(x1_train, y1_train)
var3 = ridge_CV.score(x1_train, y1_train)
var4 = ridge_CV.score(x1_test, y1_test)
print(f"\nRidgeCV modülü ile; \nEğitim verilerinin ayarlanmış R-kare değeri: {var3:.3f}")
print(f"Test verilerinin ayarlanmış R-kare değeri: {var4:.3f}")

mae2 = mean_absolute_error(y1_test, y_test_predict)
mse2 = mse(y1_test, y_test_predict)
rmse2 = rmse(y1_test, y_test_predict)
mape2 = (abs((y1_test - y_test_predict)) / y1_test).mean() * 100

print("\n---------Test Kümesi Hata İstatistikleri---------")
print(f"Ortalama Mutlak Hata (MAE)        : {mae2:.2f}")
print(f"Ortalama Kare Hata (MSE)          : {mse2:.2f}")
print(f"Kök Ortalama Kare Hata (RMSE)     : {rmse2:.2f}")
print(f"Ortalama Mutlak Yüzde Hata (MAPE) : %{mape2:.2f}")
      
y_test_predict = ridge_CV.predict(x1_test)
mae2 = mean_absolute_error(y1_test, y_test_predict)
mse2 = mse(y1_test, y_test_predict)
rmse2 = rmse(y1_test, y_test_predict)
mape2 = (abs((y1_test - y_test_predict)) / y1_test).mean() * 100

print("\nRidgeCV Modülü ile;\n---------Test Kümesi Hata İstatistikleri---------")
print(f"Ortalama Mutlak Hata (MAE)        : {mae2:.2f}")
print(f"Ortalama Kare Hata (MSE)          : {mse2:.2f}")
print(f"Kök Ortalama Kare Hata (RMSE)     : {rmse2:.2f}")
print(f"Ortalama Mutlak Yüzde Hata (MAPE) : %{mape2:.2f}")

Eğitim verilerinin ayarlanmış R-kare değeri: 0.753
Test verilerinin ayarlanmış R-kare değeri: 0.739

RidgeCV modülü ile; 
Eğitim verilerinin ayarlanmış R-kare değeri: 0.752
Test verilerinin ayarlanmış R-kare değeri: 0.819

---------Test Kümesi Hata İstatistikleri---------
Ortalama Mutlak Hata (MAE)        : 22670.35
Ortalama Kare Hata (MSE)          : 1059032860.23
Kök Ortalama Kare Hata (RMSE)     : 32542.79
Ortalama Mutlak Yüzde Hata (MAPE) : %12.92

RidgeCV Modülü ile;
---------Test Kümesi Hata İstatistikleri---------
Ortalama Mutlak Hata (MAE)        : 21679.89
Ortalama Kare Hata (MSE)          : 970012443.36
Kök Ortalama Kare Hata (RMSE)     : 31145.02
Ortalama Mutlak Yüzde Hata (MAPE) : %12.06


***
## Model 3: Lasso Regresyonu

In [6]:
cv_results3 = cross_validate(Lasso(), x1_train, y1_train, cv=5, 
                             return_train_score = True)
cv_results3

{'fit_time': array([0.00999451, 0.00699854, 0.0069952 , 0.00799274, 0.00599813]),
 'score_time': array([0.0049994 , 0.00499749, 0.00399828, 0.00300074, 0.00299931]),
 'test_score': array([0.79177309, 0.77651497, 0.7682158 , 0.77596367, 0.58034874]),
 'train_score': array([0.74367633, 0.74506856, 0.74523175, 0.74610488, 0.78621689])}

In [7]:
#lasso_reg = Lasso(alpha = 1000)
#lasso_reg.fit(x1_train, y1_train)

#y_test_predict = lasso_reg.predict(x1_test)

#print(f"Eğitim verilerinin ayarlanmış R-kare değeri: {lasso_reg.score(x1_train, y1_train):.3f}")
#print(f"Test verilerinin ayarlanmış R-kare değeri: {lasso_reg.score(x1_test, y1_test):.3f}")

y_test_predict = cross_val_predict(Lasso(), x1_test, y1_test, cv=5)

var1 = cv_results3["train_score"].mean()
var2 = cv_results3["test_score"].mean()

print(f"Eğitim verilerinin ayarlanmış R-kare değeri: {var1:.3f}")
print(f"Test verilerinin ayarlanmış R-kare değeri: {var2:.3f}")

lasso_CV = LassoCV(alphas=np.logspace(-1, 5, 7)).fit(x1_train, y1_train)
var3 = lasso_CV.score(x1_train, y1_train)
var4 = lasso_CV.score(x1_test, y1_test)
print(f"\nLassoCV modülü ile; \nEğitim verilerinin ayarlanmış R-kare değeri: {var3:.3f}")
print(f"Test verilerinin ayarlanmış R-kare değeri: {var4:.3f}")

mae3 = mean_absolute_error(y1_test, y_test_predict)
mse3 = mse(y1_test, y_test_predict)
rmse3 = rmse(y1_test, y_test_predict)
mape3 = (abs((y1_test - y_test_predict)) / y1_test).mean() * 100

print("\n---------Test Kümesi Hata İstatistikleri---------")
print(f"Ortalama Mutlak Hata (MAE)        : {mae3:.2f}")
print(f"Ortalama Kare Hata (MSE)          : {mse3:.2f}")
print(f"Kök Ortalama Kare Hata (RMSE)     : {rmse3:.2f}")
print(f"Ortalama Mutlak Yüzde Hata (MAPE) : %{mape3:.2f}")
      
y_test_predict = lasso_CV.predict(x1_test)
mae3 = mean_absolute_error(y1_test, y_test_predict)
mse3 = mse(y1_test, y_test_predict)
rmse3 = rmse(y1_test, y_test_predict)
mape3 = (abs((y1_test - y_test_predict)) / y1_test).mean() * 100

print("\nLassoCV Modülü ile;\n---------Test Kümesi Hata İstatistikleri---------")
print(f"Ortalama Mutlak Hata (MAE)        : {mae3:.2f}")
print(f"Ortalama Kare Hata (MSE)          : {mse3:.2f}")
print(f"Kök Ortalama Kare Hata (RMSE)     : {rmse3:.2f}")
print(f"Ortalama Mutlak Yüzde Hata (MAPE) : %{mape3:.2f}")

Eğitim verilerinin ayarlanmış R-kare değeri: 0.753
Test verilerinin ayarlanmış R-kare değeri: 0.739

LassoCV modülü ile; 
Eğitim verilerinin ayarlanmış R-kare değeri: 0.752
Test verilerinin ayarlanmış R-kare değeri: 0.819

---------Test Kümesi Hata İstatistikleri---------
Ortalama Mutlak Hata (MAE)        : 22667.83
Ortalama Kare Hata (MSE)          : 1058639738.78
Kök Ortalama Kare Hata (RMSE)     : 32536.74
Ortalama Mutlak Yüzde Hata (MAPE) : %12.92

LassoCV Modülü ile;
---------Test Kümesi Hata İstatistikleri---------
Ortalama Mutlak Hata (MAE)        : 21680.13
Ortalama Kare Hata (MSE)          : 970011197.46
Kök Ortalama Kare Hata (RMSE)     : 31145.00
Ortalama Mutlak Yüzde Hata (MAPE) : %12.06


***
## Model 4: ElasticNet Regresyonu

In [8]:
cv_results4 = cross_validate(ElasticNet(), x1_train, y1_train, cv=5, 
                             return_train_score = True)
cv_results4

{'fit_time': array([0.00599575, 0.00399709, 0.0039978 , 0.00499725, 0.00499439]),
 'score_time': array([0.0019989 , 0.00299811, 0.00199866, 0.00199962, 0.0019989 ]),
 'test_score': array([0.78180567, 0.76908349, 0.76810289, 0.77363189, 0.54469724]),
 'train_score': array([0.73471826, 0.73537914, 0.73493879, 0.73629079, 0.77878843])}

In [9]:
#elasticNet_reg = ElasticNet(alpha = 1000, l1_ratio=0.5)
#elasticNet_reg.fit(x1_train, y1_train)

#y_test_predict = elasticNet_reg.predict(x1_test)

#print(f"Eğitim verilerinin ayarlanmış R-kare değeri: {elasticNet_reg.score(x1_train, y1_train):.3f}")
#print(f"Test verilerinin ayarlanmış R-kare değeri: {elasticNet_reg.score(x1_test, y1_test):.3f}")

y_test_predict = cross_val_predict(ElasticNet(), x1_test, y1_test, cv=5)

var1 = cv_results4["train_score"].mean()
var2 = cv_results4["test_score"].mean()

print(f"Eğitim verilerinin ayarlanmış R-kare değeri: {var1:.3f}")
print(f"Test verilerinin ayarlanmış R-kare değeri: {var2:.3f}")

elasticNet_CV = ElasticNetCV(alphas=np.logspace(-1, 5, 7)).fit(x1_train, y1_train)
var3 = elasticNet_CV.score(x1_train, y1_train)
var4 = elasticNet_CV.score(x1_test, y1_test)
print(f"\nElasticNetCV modülü ile; \nEğitim verilerinin ayarlanmış R-kare değeri: {var3:.3f}")
print(f"Test verilerinin ayarlanmış R-kare değeri: {var4:.3f}")

mae4 = mean_absolute_error(y1_test, y_test_predict)
mse4 = mse(y1_test, y_test_predict)
rmse4 = rmse(y1_test, y_test_predict)
mape4 = (abs((y1_test - y_test_predict)) / y1_test).mean() * 100

print("\n---------Test Kümesi Hata İstatistikleri---------")
print(f"Ortalama Mutlak Hata (MAE)        : {mae4:.2f}")
print(f"Ortalama Kare Hata (MSE)          : {mse4:.2f}")
print(f"Kök Ortalama Kare Hata (RMSE)     : {rmse4:.2f}")
print(f"Ortalama Mutlak Yüzde Hata (MAPE) : %{mape4:.2f}")
      
y_test_predict = elasticNet_CV.predict(x1_test)
mae4 = mean_absolute_error(y1_test, y_test_predict)
mse4 = mse(y1_test, y_test_predict)
rmse4 = rmse(y1_test, y_test_predict)
mape4 = (abs((y1_test - y_test_predict)) / y1_test).mean() * 100

print("\nElasticNetCV Modülü ile;\n---------Test Kümesi Hata İstatistikleri---------")
print(f"Ortalama Mutlak Hata (MAE)        : {mae4:.2f}")
print(f"Ortalama Kare Hata (MSE)          : {mse4:.2f}")
print(f"Kök Ortalama Kare Hata (RMSE)     : {rmse4:.2f}")
print(f"Ortalama Mutlak Yüzde Hata (MAPE) : %{mape4:.2f}")
del var1, var2, var3, var4

Eğitim verilerinin ayarlanmış R-kare değeri: 0.744
Test verilerinin ayarlanmış R-kare değeri: 0.727

ElasticNetCV modülü ile; 
Eğitim verilerinin ayarlanmış R-kare değeri: 0.752
Test verilerinin ayarlanmış R-kare değeri: 0.819

---------Test Kümesi Hata İstatistikleri---------
Ortalama Mutlak Hata (MAE)        : 23535.63
Ortalama Kare Hata (MSE)          : 1126757223.73
Kök Ortalama Kare Hata (RMSE)     : 33567.20
Ortalama Mutlak Yüzde Hata (MAPE) : %13.45

ElasticNetCV Modülü ile;
---------Test Kümesi Hata İstatistikleri---------
Ortalama Mutlak Hata (MAE)        : 21568.01
Ortalama Kare Hata (MSE)          : 968202287.90
Kök Ortalama Kare Hata (RMSE)     : 31115.95
Ortalama Mutlak Yüzde Hata (MAPE) : %12.00


***
# Yorumlar:
* Test verileri için **Ayarlanmış R-kare** Değerleri:
    * ElasticNet: 0.727
    * Ridge & Lasso: 0.739
    * OLS: 0.776
    * RidgeCV, LassoCV & ElasticNetCV: 0.819 <br>


* Test Kümesi, **Ortalama Mutlak Yüzde Hata** Değerleri:
    * OLS: %15.08
    * ElasticNet: %13.45
    * Ridge & Lasso: %12.92
    * RidgeCV & LassoCV: %12.06
    * ElasticNetCV: %12.00


En iyi modelin, ElasticNetCV modülü ile oluşturulan ElasticNet regresyon modeli olduğunu görürüz.

## \## Sorular:
* cross_validate ile yapılan ve .mean() alınan değerler ile xyzCV modülüyle (örn: LassoCV) bulunan değerlerin farklı olmasının sebebi, alpha değeri mi? cross_validate'te alpha girilen bir yer yok çünkü. 
* Ridge ve Lasso regresyon değerlerinin neredeyse aynı çıkmasındaki sebep nedir?
* Ayarlanmış R-kare değerinin, xyzCV modülleri hariç OLS'de en yüksek çıkması garip geldi. <br>Alpha değerlerini elle ayarladığımız **lasso_reg = Lasso(alpha = 1000)** yönteminde değerler OLS'den daha yüksek çıkmıştı. Sanırım cross_validate yaparken girdiği alpha değerinden kaynaklanıyor bu. El ile ayarlayınca veya xyzCV modüllerinde daha yüksek başarıya sahip. Bu yorumum yanlış değildir sanırım :) Ayrıca ilk soruyu cevaplamış oldum galiba.
* Kodlarda yanlış yaptığım bir yerler var mı? Yanlış yaptığım yöntemler vb.?
* Başka ne tür iyileştirmeler yapabiliriz? Sanırım fold değerlerini 5'ten yukarı ayarlayabilir, arttırabiliriz değil mi?